In [7]:
import ray
import time

#ray.init()

@ray.remote
class SomeActor():

    def function(self):
        print("Running in sync function")
        time.sleep(2)
        print("Done in sync function")

    async def function_async(self):
        print("Running in sync function")
        await asyncio.sleep(2)
        print("Done in sync function")

    def terminate(self):
        print("Self-killing")
        ray.actor.exit_actor()

for i in range(2):
    try:
        actor = SomeActor.remote()
        second = actor.function.remote()
        ray.get(actor.terminate.remote())
        print("waiting for function")
        result = ray.get(second)
        print("done")
    except Exception as e:
        print(f"raised {e}")

(pid=9244) Running in sync function
(pid=9244) Done in sync function
(pid=9244) Self-killing
raised The actor died unexpectedly before finishing this task.
(pid=9272) Running in sync function
(pid=9272) Done in sync function
(pid=9272) Self-killing
raised The actor died unexpectedly before finishing this task.


In [2]:
import ray
import time

#ray.init()


class SomeActor():

    def function(self):
        print("Running in sync function")
        time.sleep(2)
        print("Done in sync function")

    async def function_async(self):
        print("Running in sync function")
        await asyncio.sleep(2)
        print("Done in sync function")

    def terminate(self):
        print("Self-killing")

for i in range(2):
    try:
        actor = SomeActor.remote()
        second = actor.function.remote()
        ray.get(actor.terminate.remote())
        print("waiting for function")
        result = ray.get(second)
        print("done")
    except Exception as e:
        print(f"raised {e}")

/Users/smanthony/opt/anaconda3/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 52051 instead
  http_address["port"], self.http_server.port


raised This event loop is already running
raised This event loop is already running


Task exception was never retrieved
future: <Task finished coro=<test() done, defined at <ipython-input-2-ad645f6adfb1>:25> exception=AttributeError("'function' object has no attribute 'function_async'")>
Traceback (most recent call last):
  File "<ipython-input-2-ad645f6adfb1>", line 26, in test
    fut = kill.function_async().result()
AttributeError: 'function' object has no attribute 'function_async'
Task exception was never retrieved
future: <Task finished coro=<test() done, defined at <ipython-input-2-ad645f6adfb1>:25> exception=AttributeError("'function' object has no attribute 'function_async'")>
Traceback (most recent call last):
  File "<ipython-input-2-ad645f6adfb1>", line 26, in test
    fut = kill.function_async().result()
AttributeError: 'function' object has no attribute 'function_async'


In [1]:
#this is for the actor handles


In [6]:
import ray
import time
#ray.init()
@ray.remote
class Counter(object):
    def __init__(self):
        self.value = 0

    def increment(self):
        self.value += 1
        return self.value

    def get_counter(self):
        return self.value
    
@ray.remote
def f(counter):
    for _ in range(1000):
        time.sleep(0.1)
        counter.increment.remote()
        
counter = Counter.remote()
print(counter)
# Start some tasks that use the actor.
y = [f.remote(counter) for _ in range(3)]
print(y)

# Print the counter value.
for _ in range(10):
    time.sleep(1)
    print(ray.get(counter.get_counter.remote()))

Actor(Counter,6cdd55a130cc605b632cafbb01000000)
[ObjectRef(b4bba3bd025ddc5effffffffffffffffffffffff0100000001000000), ObjectRef(d5a75db31f99bd73ffffffffffffffffffffffff0100000001000000), ObjectRef(d68fec326c8433c9ffffffffffffffffffffffff0100000001000000)]
18
44
72
102
131
159
189
218
246
276


In [4]:
import dask
import time
class Counter(object):
    def __init__(self):
        self.value = 0
   
    def increment(self):
        self.value += 1
        return self.value
    
    def get_counter(self):
        return self.value
    
def f(counter):
    for _ in range(1000):
        time.sleep(0.1)
        counter.increment().result()
    
from dask.distributed import Client

client = Client()

# Create 10 Actors, and call increment() once on each of them
alist = [client.submit(Counter, actor=True).result() for _ in range(3)]
results = [f(a) for a in alist]

# for _ in range(10):
#     time.sleep(1)
# #     print(dask.compute(counter.get_counter().result()))

#results = [c.f().result() for c in counters]
# print(results)

# # Call increment() 5 times on the first Actor in the list
# results = [counters[0].increment().result() for _ in range(5)]
# print(results)